In [65]:
import glob
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import pandas as pd

def get_exif_data(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    exif_dict = {}
    exif_dict['ImageFilename'] = image_path.split("\\")[1]
    if exif_data is not None:
        for key, value in exif_data.items():
            if key in TAGS:
                tag = TAGS[key]
                if tag == 'GPSInfo':
                    gps_data = {}
                    for t in value:
                        sub_tag = GPSTAGS.get(t, t)
                        if str( value[t])[0]=='b':
                            gps_data[sub_tag] =  value[t].decode('ascii')
                        else:
                            gps_data[sub_tag] = value[t]
                    exif_dict.update(zip(list(gps_data.keys()), list(gps_data.values())))
                else:
                    if str(value)[0]=='b':
                        exif_dict[tag] = value.decode('ascii')
                    else:
                        exif_dict[tag] = value
    return exif_dict


if __name__ == "__main__":
    image_files = glob.glob('data/*.jpg')
    all_exif_data = []
    
    for image_path in image_files:
        exif_data = get_exif_data(image_path)  
        all_exif_data.append(exif_data)
    
    df = pd.DataFrame(all_exif_data)
    df.to_csv('data/exif_metadata.csv', index=False)